In [1]:
from AutoML import AutoML, DataNet, AutoMLDataset
#DataNet().restart_datanodes()
classes_semantics = [113988, 113989, 113990]
dataset = AutoMLDataset(name="person_classification")#, semantics=classes_semantics, crops=True) #, override=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

In [2]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2021-01-12 17:50:07,085	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
2021-01-12 17:50:07,086	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


In [3]:
add_images_stage = 5000

In [4]:
# get the indexes to emulate the labeled set

import numpy as np
import random

list_classes = list(dataset.dataset_header['train_images'].keys())
nb_classes = len(list_classes)        
classes={}
files = []
labels = []
nb_img_per_class = np.zeros(nb_classes, dtype = np.int32)
classes_cardinality = {}

for class_name in list_classes:
    class_index = list_classes.index(class_name)
    classes_cardinality[class_name] = len(dataset.dataset_header['train_images'][class_name])
    nb_img_per_class[class_index] = classes_cardinality[class_name]
    start = len(files)
    id_files = dataset.dataset_header["train_images"][class_name]
    files+=id_files
    labels+=[class_index] * len(id_files)
    end = len(files)
    classes[class_index]=range(start, end)

    
all_files = np.array(files)
all_labels = np.array(labels)
indices = list(range(len(files)))
random.seed(101)
random.shuffle(indices)
labeled_set_index = indices[:add_images_stage ]
unlabeled_set_index = indices[add_images_stage :]

files_labeled_set = list(all_files[labeled_set_index])
files_unlabeled_set = list(all_files[unlabeled_set_index])

labels_labeled_set = list(all_labels[labeled_set_index])
labels_unlabeled_set = list(all_labels[unlabeled_set_index])


In [5]:
# load defaulft config
import yaml

config_path = '/mnt/Ressources/Andres/Temp_active/configs/Datanet_AL_2.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

print(config)

{'DATASET': {'Data_augementation': True, 'original_size': 256, 'pad': False, 'random_crop': True, 'random_crop_pad': 30, 'random_flip': True, 'random_greyscale': False, 'random_hue': False, 'rot90': False, 'random_brightness': False, 'random_saturation': False}, 'NETWORK': {'Backbone': 'ResNet50', 'INPUT_SIZE': 256, 'MARGIN': 1.0, 'embedding_size': 128}, 'PROJECT': {'dataset_name': 'person_classification', 'group': 'Active_Learning_2', 'group_dir': '/mnt/Ressources/Andres/runs/Active_Learning_2', 'project': 'Active_Learning_Datanet', 'project_dir': '/mnt/Ressources/Andres/runs'}, 'TEST': {'batch_size': 16}, 'TRAIN': {'EPOCH_SLIT': 80, 'EPOCH_WARMUP': 2, 'EPOCH_WHOLE': 120, 'MILESTONES': [140, 180], 'batch_size': 32, 'lr': 0.0025, 'test_each': 5, 'transfer_weight_path': '/mnt/Ressources/Andres/runs/imagenet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', 'weight_lossnet_loss': 1.0, 'weight_decay': 2e-05, 'apply_weight_decay': False}}


In [6]:
import time
import os
import pandas as pd

In [7]:
from train_agent_classification_AL import Active_Learning_train
from inference_agent_classification import Active_Learning_inference

In [8]:
resumeat = 0

In [ ]:
for num_run in range(resumeat,10):

    prev_num_run = num_run-1
    if num_run==0:
        resume_model_path = False
    else:
        resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(prev_num_run),'checkpoints','checkpoint.200.hdf5')
        
    Active_Learning = Active_Learning_train.remote(config, 
                                            files_labeled_set,
                                            labels_labeled_set,
                                            classes_semantics,
                                            num_run=num_run,
                                            resume_model_path=resume_model_path,
                                            resume = False)
    
    Active_Learning.start_training.remote()
    
    # Wait util the model is training
    while True:
        time.sleep(10)
        try:
            progress_id = Active_Learning.isTraining.remote()
            response = ray.get(progress_id)
            break
        except:
            pass
        
    # wait until the model finish training
    while True:
        time.sleep(10)
        progress_id = Active_Learning.isTraining.remote()
        response = ray.get(progress_id)
        
        get_progress = Active_Learning.get_progress.remote()
        print(ray.get(get_progress))
        
        if not response:
            break
    
    del Active_Learning

    resume_model_path = os.path.join(config['PROJECT']['group_dir'],'Stage_'+str(num_run),'checkpoints','checkpoint.200.hdf5')
    AL_inference = Active_Learning_inference.remote(config , files_unlabeled_set, labels_unlabeled_set, classes_semantics, num_run, resume_model_path)
    AL_inference.inference.remote()

    run_dir   = os.path.join(config["PROJECT"]["group_dir"],"Stage_"+str(num_run))
    ordered_indexes   = os.path.join(run_dir, "ordered_indexes.csv")

    # wait the file qith the scores is generated
    while True:
        time.sleep(10)
        if os.path.isfile(ordered_indexes):
            break
    try:
        AL_inference.__ray_terminate__.remote()
    except:
        print("Here")
        
    del AL_inference

    # read the scores file and create the new labeled set and unlabeled set to repeat the trainig
    pd_ordered_indexes = pd.read_csv(ordered_indexes)
    
    new_files_set = pd_ordered_indexes['files'].values
    new_files_set = [i[2:-1] for i in new_files_set]
    new_labels_set = pd_ordered_indexes['labels'].values
    
    files_labeled_set    += list(new_files_set[:add_images_stage])
    labels_labeled_set   += list(new_labels_set[:add_images_stage])

    files_unlabeled_set  = list(new_files_set[add_images_stage:])
    labels_unlabeled_set = list(new_labels_set[add_images_stage:])

(pid=10956) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10956)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=10956) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10956)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=10956) /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=10956)   _np_qint16 = np.dtyp

(pid=10956) Subset set to : train
(pid=10956) ['Partially Dressed', 'Dressed', 'Explicit Dressed']
(pid=10956) Train_Stage_0 The backbone is:  ResNet50


(pid=10956) WARNING:tensorflow:From /mnt/Ressources/Andres/Temp_active/data_pipeline/Classification_dataset_AL.py:131: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
(pid=10956) Instructions for updating:
(pid=10956) tf.py_func is deprecated in TF V2. Instead, use
(pid=10956)     tf.py_function, which takes a python function which manipulates tf eager
(pid=10956)     tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
(pid=10956)     an ndarray (just call tensor.numpy()) but having access to eager tensors
(pid=10956)     means `tf.py_function`s can use accelerators such as GPUs as well as
(pid=10956)     being differentiable using a gradient tape.
(pid=10956)     
(pid=10956) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future v

(pid=10956) Train_Stage_0 (transfer learning) Loading weigths by name from:  /mnt/Ressources/Andres/runs/imagenet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


(pid=10956) 2021-01-12 17:50:19,993	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379
(pid=10956) 2021-01-12 17:50:19,993	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.
(pid=10956) WARNING:tensorflow:Output "Embedding" missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to "Embedding".
(pid=10956) WARNING:tensorflow:Output "Embedding" missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to "Embedding".


(pid=10956) Train_Stage_0 Init done
(pid=10956) Train_Stage_0 Start training


(pid=10956) WARNING:tensorflow:From /home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=10956) Instructions for updating:
(pid=10956) Use tf.cast instead.


{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 1, 'Total Steps': 31400, 'Progress': 0.0, 'steps_per_epoch': 157, 'status': 'Idle'}
{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 1, 'Total Steps': 31400, 'Progress': 0.0, 'steps_per_epoch': 157, 'status': 'Training'}
(pid=10956) Epoch 1/200
{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 1, 'Total Steps': 31400, 'Progress': 0.0, 'steps_per_epoch': 157, 'status': 'Training'}


(pid=10956) 2021-01-12 17:50:51.196002: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 33, 'Total Steps': 31400, 'Progress': 0.11, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 80, 'Total Steps': 31400, 'Progress': 0.25, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 1, 'Total Epoch': 200, 'Current Step': 127, 'Total Steps': 31400, 'Progress': 0.4, 'steps_per_epoch': 157, 'status': 'Training'}
(pid=10956)  - 46s - loss: 1.2346 - c_pred_loss: 0.8469 - l_pred_w_loss: 0.3877 - l_pred_s_loss: 0.3877 - c_pred_sparse_categorical_accuracy: 0.7542
(pid=10956) Epoch 2/200
{'Current Epoch': 0, 'Total Epoch': 200, 'Current Step': 162, 'Total Steps': 31400, 'Progress': 0.52, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 0, 'Total Epoch': 200, 'Current Step': 209, 'Total Steps': 31400, 'Progress': 0.66, 'steps_per_epoch': 157, 'status': 'Training'}
{'Current Epoch': 0, 'Total Epoch': 200, 'Current Step': 256, 'Total Steps': 31400, 'Progr